In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install adapter-transformers
!pip install datasets
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 70.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 104.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 16.9 MB/s eta 0:00:

In [3]:
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import json
from transformers import AutoTokenizer, AdapterConfig, AutoAdapterModel, AutoConfig
from transformers import TrainingArguments, AdapterTrainer, EvalPrediction, TrainerCallback
from transformers import AutoModelForSequenceClassification, Trainer
from transformers import EarlyStoppingCallback, IntervalStrategy, DataCollatorForLanguageModeling
from datasets import Dataset, DatasetDict

from tqdm import tqdm

import numpy as np
from datasets import concatenate_datasets, load_metric
import evaluate
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

In [9]:
df_subtask_1_en = pd.read_csv("/content/drive/MyDrive/autextification/data/subtask_1/en/train.tsv", sep='\t')
df_subtask_1_en=df_subtask_1_en.drop(df_subtask_1_en.columns[0], axis=1)

In [10]:
mapping = {
    # 0:"generated",
    # 1:"generated",
    "generated":0,
    "human":1
}
df_subtask_1_en["label"] = df_subtask_1_en['label'].map(mapping)

In [12]:
df_subtask_1_en.head()

,text,label
0,you need to stop the engine and wait until it ...,0
1,The Commission shall publish the report; an in...,0
2,"I have not been tweeting a lot lately, but I d...",0
3,I pass my exam and really thankgod for that bu...,1
4,The template will have 3 parts: a mustache sha...,1


In [13]:
df_subtask_1_en.label.unique()

array([0, 1])

In [14]:
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(df_subtask_1_en, test_size=0.1, random_state=42)

print(len(dataset_train))
print(len(dataset_test))

30460
3385


In [15]:
dataset_train.head()

,text,label
30793,Just had the best hot chocolate in Soho at a l...,0
15268,Member States shall ensure that any such refer...,0
963,"For example, if the version of Wordpress on yo...",0
15538,Time to go buy a home with this one. The lot i...,0
26241,"Workrelated coding, a vision doc, an interns p...",1


In [16]:
dataset_test.head()

,text,label
29163,The authorization granted in Article 1 shall a...,1
28479,@hundredreasons tho nothing could get better I...,0
14508,If you are going to be hosting multiple specie...,0
32961,It is made from a mixture of whole milk and re...,0
29586,The type of business you choose to form determ...,1


In [18]:
# language_model = "distilbert-base-cased"
language_model = "prajjwal1/bert-tiny"


tokenizer = AutoTokenizer.from_pretrained(language_model)

dataset_train = Dataset.from_pandas(dataset_train)
dataset_test = Dataset.from_pandas(dataset_test)

def encode_batch(batch):
    """Encodes a batch of input data using the model tokenizer."""
    return tokenizer(batch["text"], max_length=80, truncation=True, padding="max_length")


dataset_train = dataset_train.rename_column("label", "labels")
dataset_train = dataset_train.map(encode_batch, batched=True)
dataset_train.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

dataset_test = dataset_test.map(encode_batch, batched=True)
dataset_test = dataset_test.rename_column("label", "labels")
dataset_test.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/30460 [00:00<?, ? examples/s]

Map:   0%|          | 0/3385 [00:00<?, ? examples/s]

In [19]:
dataset_train.to_pandas().head()

,text,labels,__index_level_0__,input_ids,token_type_ids,attention_mask
0,Just had the best hot chocolate in Soho at a l...,0,30793,"[101, 2074, 2018, 1996, 2190, 2980, 7967, 1999...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,Member States shall ensure that any such refer...,0,15268,"[101, 2266, 2163, 4618, 5676, 2008, 2151, 2107...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,"For example, if the version of Wordpress on yo...",0,963,"[101, 2005, 2742, 1010, 2065, 1996, 2544, 1997...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,Time to go buy a home with this one. The lot i...,0,15538,"[101, 2051, 2000, 2175, 4965, 1037, 2188, 2007...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,"Workrelated coding, a vision doc, an interns p...",1,26241,"[101, 2147, 16570, 4383, 16861, 1010, 1037, 44...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


## Modelling

In [20]:
model = AutoModelForSequenceClassification.from_pretrained(language_model)
  
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=3,
    seed = 42,
    output_dir="./training_output",
    # label_names=["generated", "human"]
    # per_device_train_batch_size=64,
    # per_device_eval_batch_size=64,
    # dataloader_num_workers=32,
    logging_steps=100,
    # save_total_limit = 2,
    # overwrite_output_dir=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    # load_best_model_at_end=True,
    # report_to='tensorboard',
    # metric_for_best_model='accuracy'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_test,
    compute_metrics=compute_metrics,
    # data_collator=data_collator,

    # callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
)

trainer.train()

t_metrics = trainer.evaluate(dataset_test)

Some weights of the model checkpoint at prajjwal1/bert-tiny were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initia

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.381400,0.397430,0.828656,0.828422,0.828369,0.828484
2,0.359900,0.426332,0.838109,0.837298,0.839635,0.836627
3,0.264200,0.552737,0.841064,0.839961,0.844101,0.839097


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, text. If __index_level_0__, text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3385
  Batch size = 8
Saving model checkpoint to ./training_output/checkpoint-3808
Configuration saved in ./training_output/checkpoint-3808/config.json
Model weights saved in ./training_output/checkpoint-3808/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, text. If __index_level_0__, text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3385
  Batch size = 8
Saving model checkpoint to ./training_output/checkpoint-7616
Co